In [2]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import pandas as pd
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import leaguegamefinder, boxscoretraditionalv2
from nba_api.stats.endpoints import shotchartdetail
import time


In [5]:
#Ejemplo y prueba de uso de la API-nba
# Obtener todos los jugadores
all_players = players.get_players()

# Buscar un jugador específico, por ejemplo, LeBron James
lebron = [player for player in all_players if player['full_name'] == 'LeBron James'][0]

# Obtener información común sobre el jugador
player_id = lebron['id']
player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)

# Convertir la información a un DataFrame
player_info_df = player_info.get_data_frames()[0]

print(player_info_df)

# Guardar la información en un archivo CSV
player_info_df.to_csv('lebron_james_info.csv', index=False)

   PERSON_ID FIRST_NAME LAST_NAME DISPLAY_FIRST_LAST DISPLAY_LAST_COMMA_FIRST  \
0       2544     LeBron     James       LeBron James            James, LeBron   

  DISPLAY_FI_LAST   PLAYER_SLUG            BIRTHDATE  \
0        L. James  lebron-james  1984-12-30T00:00:00   

                         SCHOOL COUNTRY  ...    PLAYERCODE FROM_YEAR TO_YEAR  \
0  St. Vincent-St. Mary HS (OH)     USA  ...  lebron_james      2003    2024   

   DLEAGUE_FLAG NBA_FLAG GAMES_PLAYED_FLAG DRAFT_YEAR DRAFT_ROUND  \
0             N        Y                 Y       2003           1   

   DRAFT_NUMBER GREATEST_75_FLAG  
0             1                Y  

[1 rows x 33 columns]


In [2]:

# Función para hacer solicitudes con manejo de rate limit
def make_request_with_rate_limiting(endpoint, params=None):
    while True:
        try:
            response = endpoint(**params)
            return response.get_data_frames()[0]
        except Exception as e:
            print(f"Error: {e}. Reintentando en 60 segundos...")
            time.sleep(60)

# Lista para almacenar datos
all_seasons_stats = []

# Temporadas desde 2015-16 hasta 2021-22
seasons = ['2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22']

# Iterar sobre cada temporada y obtener estadísticas
for season in seasons:
    print(f"Procesando temporada: {season}")
    params = {
        'season': season,
        'season_type_all_star': 'Regular Season',
    }
    season_stats = make_request_with_rate_limiting(leaguedashplayerstats.LeagueDashPlayerStats, params)
    season_stats['SEASON'] = season  # Añadir la columna de temporada
    all_seasons_stats.append(season_stats)

# Concatenar todas las estadísticas en un DataFrame
all_player_stats = pd.concat(all_seasons_stats, ignore_index=True)

# Guardar las estadísticas en un archivo CSV
all_player_stats.to_csv('player_average_stats_2015_2022.csv', index=False)

print("Estadísticas guardadas en 'player_average_stats_2015_2022.csv'")


Procesando temporada: 2015-16
Procesando temporada: 2016-17
Procesando temporada: 2017-18
Procesando temporada: 2018-19
Procesando temporada: 2019-20
Procesando temporada: 2020-21
Procesando temporada: 2021-22
Estadísticas guardadas en 'player_average_stats_2015_2022.csv'


In [3]:

# Función para hacer solicitudes con manejo de rate limiting
def make_request_with_rate_limiting(endpoint, params=None):
    while True:
        try:
            response = endpoint(**params)
            return response.get_data_frames()[0]
        except Exception as e:
            print(f"Error: {e}. Reintentando en 60 segundos...")
            time.sleep(60)

# Lista para almacenar datos
all_game_stats = []

# Temporadas desde 2015-16 hasta 2021-22
seasons = ['2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22']
season_types = ['Regular Season', 'Playoffs']

# Iterar sobre cada temporada y obtener la lista de juegos
for season in seasons:
    for season_type in season_types:
        print(f"Procesando {season_type} de la temporada: {season}")
        game_finder_params = {
            'season_nullable': season,
            'season_type_nullable': season_type
        }
        game_finder = make_request_with_rate_limiting(leaguegamefinder.LeagueGameFinder, game_finder_params)
        games = game_finder['GAME_ID'].unique()
        
        # Obtener las estadísticas de cada juego
        for game_id in games:
            print(f"Procesando juego ID: {game_id}")
            boxscore_params = {'game_id': game_id}
            game_stats = make_request_with_rate_limiting(boxscoretraditionalv2.BoxScoreTraditionalV2, boxscore_params)
            game_stats['GAME_ID'] = game_id
            game_stats['SEASON'] = season  # Añadir la columna de temporada
            game_stats['SEASON_TYPE'] = season_type  # Añadir la columna de tipo de temporada
            all_game_stats.append(game_stats)

# Concatenar todas las estadísticas en un DataFrame
all_game_player_stats = pd.concat(all_game_stats, ignore_index=True)

# Guardar las estadísticas en un archivo CSV
all_game_player_stats.to_csv('player_game_stats_2015_2022_detailed.csv', index=False)

print("Estadísticas guardadas en 'player_game_stats_2015_2022_detailed.csv'")


Procesando Regular Season de la temporada: 2015-16
Procesando juego ID: 0021501222
Procesando juego ID: 0021501224
Procesando juego ID: 0021501228
Procesando juego ID: 0021501223
Procesando juego ID: 0021501227
Procesando juego ID: 0021501220
Procesando juego ID: 0021501225
Procesando juego ID: 0021501217
Procesando juego ID: 0021501230
Procesando juego ID: 0021501226
Procesando juego ID: 0021501219
Procesando juego ID: 0021501229
Procesando juego ID: 0021501221
Procesando juego ID: 0021501218
Procesando juego ID: 0021501212
Procesando juego ID: 0021501215
Procesando juego ID: 0021501214
Procesando juego ID: 0021501213
Procesando juego ID: 0021501216
Procesando juego ID: 0021501209
Procesando juego ID: 0021501211
Procesando juego ID: 0021501210
Procesando juego ID: 0021501208
Procesando juego ID: 0021501204
Procesando juego ID: 0021501205
Procesando juego ID: 0021501206
Procesando juego ID: 0021501207
Procesando juego ID: 0021501203
Procesando juego ID: 0021501200
Procesando juego ID: 

C:\Users\Rembert\AppData\Local\Temp\ipykernel_7940\899356165.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_game_player_stats = pd.concat(all_game_stats, ignore_index=True)


Estadísticas guardadas en 'player_game_stats_2015_2022_detailed.csv'


In [5]:
# Función para obtener la lista de todos los jugadores activos en la temporada actual
def get_active_players(season):
    players = commonallplayers.CommonAllPlayers(is_only_current_season=1, season=season).get_data_frames()[0]
    return players[['PERSON_ID', 'DISPLAY_FIRST_LAST']]

# Función para obtener el team_id de un jugador
def get_team_id(player_id):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()[0]
    team_id = player_info.loc[0, 'TEAM_ID']
    return team_id

# Función para obtener los detalles de los tiros de un jugador para una temporada
def get_shot_chart_details(player_id, team_id, season):
    shot_chart = shotchartdetail.ShotChartDetail(player_id=player_id, team_id=team_id, season_nullable=season, season_type_all_star='Regular Season')
    return shot_chart.get_data_frames()[0]

# Lista de temporadas a considerar
seasons = ['2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22']

# DataFrame para almacenar todos los datos de tiros
all_shot_data = pd.DataFrame()

# Obtener la lista de jugadores activos en la temporada más reciente
active_players = get_active_players('2022-23')

# Iterar sobre cada temporada y cada jugador activo
for season in seasons:
    for index, player in active_players.iterrows():
        player_id = player['PERSON_ID']
        player_name = player['DISPLAY_FIRST_LAST']
        print(f"Obteniendo datos de tiros para {player_name} en la temporada {season}")

        try:
            # Obtener el team_id del jugador
            team_id = get_team_id(player_id)
            
            # Obtener los detalles de los tiros
            shot_data = get_shot_chart_details(player_id, team_id, season)
            shot_data['PLAYER_NAME'] = player_name
            shot_data['SEASON'] = season

            # Añadir los datos al DataFrame general
            all_shot_data = pd.concat([all_shot_data, shot_data], ignore_index=True)
            
            # Pausa para evitar sobrecargar la API
            time.sleep(1)
        except Exception as e:
            print(f"Error al obtener datos para {player_name}: {e}")

# Guardar los datos en un archivo CSV
all_shot_data.to_csv('shot_chart_details_2015_2022.csv', index=False)

print("Datos de tiros guardados en 'shot_chart_details_2015_2022.csv'")


Obteniendo datos de tiros para Steven Adams en la temporada 2015-16
Obteniendo datos de tiros para D.J. Augustin en la temporada 2015-16
Obteniendo datos de tiros para Will Barton en la temporada 2015-16
Obteniendo datos de tiros para Khem Birch en la temporada 2015-16
Obteniendo datos de tiros para Leandro Bolmaro en la temporada 2015-16
Obteniendo datos de tiros para James Bouknight en la temporada 2015-16
Obteniendo datos de tiros para Tony Bradley en la temporada 2015-16
Obteniendo datos de tiros para Jarrell Brantley en la temporada 2015-16
Obteniendo datos de tiros para Sterling Brown en la temporada 2015-16
Obteniendo datos de tiros para Deonte Burton en la temporada 2015-16
Obteniendo datos de tiros para John Butler Jr. en la temporada 2015-16
Obteniendo datos de tiros para Facundo Campazzo en la temporada 2015-16
Obteniendo datos de tiros para Vlatko Cancar en la temporada 2015-16
Obteniendo datos de tiros para Vernon Carey Jr. en la temporada 2015-16
Obteniendo datos de tiros

In [3]:
# Función para obtener los partidos jugados en una temporada
def get_games_for_season(season):
    game_finder = leaguegamefinder.LeagueGameFinder(season_nullable=season)
    games = game_finder.get_data_frames()[0]
    return games

# Lista de temporadas a considerar
seasons = ['2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22']

# DataFrame para almacenar todos los datos de partidos
all_games_data = pd.DataFrame()

# Iterar sobre cada temporada y obtener los partidos
for season in seasons:
    games_season = get_games_for_season(season)
    all_games_data = pd.concat([all_games_data, games_season], ignore_index=True)

# Mostrar los primeros registros para verificar
print(all_games_data.head())

# Guardar los datos en un archivo CSV
all_games_data.to_csv('calendario_partidos_2015_2022.csv', index=False)

print("Datos de partidos guardados en 'calendario_partidos_2015_2022.csv'")

  SEASON_ID     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME     GAME_ID  \
0     42015  1610612744               GSW  Golden State Warriors  0041500407   
1     42015  1610612739               CLE    Cleveland Cavaliers  0041500407   
2     42015  1610612739               CLE    Cleveland Cavaliers  0041500406   
3     42015  1610612744               GSW  Golden State Warriors  0041500406   
4     42015  1610612744               GSW  Golden State Warriors  0041500405   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2016-06-19  GSW vs. CLE  L  239   89  ...   0.769     7    32   39   22   
1  2016-06-19    CLE @ GSW  W  241   93  ...   0.840     9    39   48   17   
2  2016-06-16  CLE vs. GSW  W  240  115  ...   0.781     8    37   45   24   
3  2016-06-16    GSW @ CLE  L  238  101  ...   0.690     9    26   35   19   
4  2016-06-13  GSW vs. CLE  L  242   97  ...   0.731    13    30   43   18   

   STL  BLK  TOV  PF  PLUS_MINUS  
0    7    5   1